In [462]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [463]:
dataset_prime = pd.read_csv('/content/drive/MyDrive/Deep Learning/HW1/agaricus-lepiota.data')
print(dataset_prime.shape)
dataset_prime.head()

(8123, 23)


,p,x,s,n,t,p.1,f,c,n.1,k,e,e.1,s.1,s.2,w,w.1,p.2,w.2,o,p.3,k.1,s.3,u
0,e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
1,e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
2,p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
3,e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g
4,e,x,y,y,t,a,f,c,b,n,e,c,s,s,w,w,p,w,o,p,k,n,g


In [464]:
dataset = dataset_prime.copy()


In [465]:
print(np.unique(dataset['e.1']))
print(np.unique(dataset['p.2']))

['?' 'b' 'c' 'e' 'r']
['p']


In [466]:

dataset = dataset.drop(columns=['e.1'])
print(dataset.shape)

(8123, 22)


In [467]:
# for col in dataset:
#   if len(np.unique(dataset[col])) == 1:
#     dataset = dataset.drop(columns=[col])

In [468]:
dataset.shape

(8123, 22)

In [469]:
for col in dataset:

  # create dictionary of ordinal to integer mapping
  dict_col = {}
  i = 0
  for el in np.unique(dataset[col]):
    dict_col[el] = i 
    i+=1
  # apply using map
  dataset[col] = dataset[col].map(dict_col)
  if col != 'p':
    dataset[col] = dataset[col]/ 2.6
    
    # dataset[col] = (dataset[col] - dataset[col].min(axis=0)) / (dataset[col].max(axis=0) - dataset[col].min(axis=0))

dataset.head()

,p,x,s,n,t,p.1,f,c,n.1,k,e,s.1,s.2,w,w.1,p.2,w.2,o,p.3,k.1,s.3,u
0,0,1.923077,0.769231,3.461538,0.384615,0.000000,0.384615,0.000000,0.000000,1.538462,0.000000,0.769231,0.769231,2.692308,2.692308,0.0,0.769231,0.384615,1.538462,1.153846,0.769231,0.384615
1,0,0.000000,0.769231,3.076923,0.384615,1.153846,0.384615,0.000000,0.000000,1.923077,0.000000,0.769231,0.769231,2.692308,2.692308,0.0,0.769231,0.384615,1.538462,1.153846,0.769231,1.153846
2,1,1.923077,1.153846,3.076923,0.384615,2.307692,0.384615,0.000000,0.384615,1.923077,0.000000,0.769231,0.769231,2.692308,2.692308,0.0,0.769231,0.384615,1.538462,0.769231,1.153846,1.923077
3,0,1.923077,0.769231,1.153846,0.000000,1.923077,0.384615,0.384615,0.000000,1.538462,0.384615,0.769231,0.769231,2.692308,2.692308,0.0,0.769231,0.384615,0.000000,1.153846,0.000000,0.384615
4,0,1.923077,1.153846,3.461538,0.384615,0.000000,0.384615,0.000000,0.000000,1.923077,0.000000,0.769231,0.769231,2.692308,2.692308,0.0,0.769231,0.384615,1.538462,0.769231,0.769231,0.384615


In [470]:

msk = np.random.rand(len(dataset)) < 0.7

train = dataset[msk]

test = dataset[~msk]

In [471]:
test.shape

(2509, 22)

In [472]:
y_train = train['p']
X_train = train.drop(columns=['p'])
y_test = test['p']
X_test = test.drop(columns=['p'])

In [473]:
def prior_probability(y):
  unique, counts = np.unique(y_train, return_counts=True)
  class_counts = dict(zip(unique, counts))
  
  prior = {}
  for label in class_counts:
    prior[label] = float(class_counts[label]) / sum(class_counts.values())
  return prior

In [474]:
def calc_mean(X,y):
  mean_features_class = {}
  
  labels = np.unique(y)
  for label in labels:
    
    X_class = X[y == label]
    
    mean_features = {}
    for feature in X_class:
      mean_features[feature] = np.mean(X_class[feature])

    
    mean_features_class[label] = mean_features
  return mean_features_class

In [475]:
def calc_variance(X,y):
  var_features_class = {}
  
  labels = np.unique(y)
  for label in labels:
    
    X_class = X[y == label]
    
    var_features = {}
    for feature in X_class:
      var_features[feature] = np.var(X_class[feature])

    
    var_features_class[label] = var_features
  return var_features_class

In [476]:
def feature_prob(feature_value, mean, variance):
  prob = (1./math.sqrt(2 * math.pi * variance)) * math.exp(-1 * ((feature_value - mean)**2) / (2. * variance))
  return prob

In [477]:
var_feature_class = calc_variance(X_train,y_train)
mean_feature_class = calc_mean(X_train,y_train)
prior_class = prior_probability(y_train)
def naive_bayes(sample):
  # var_feature_class = calc_variance(X,y)
  # mean_feature_class = calc_mean(X,y)
  # prior_class = prior_probability(y)
  labels = prior_class.keys()
  features = var_feature_class[list(labels)[0]].keys()
  posterior = {}
  for label in labels:
    posterior[label] = prior_class[label]
    for feature in features:
      if var_feature_class[label][feature] != 0:
        posterior[label] *= feature_prob(sample[feature], mean_feature_class[label][feature], var_feature_class[label][feature])
      else:
        posterior[label] *= 1
  for label in posterior:
    posterior[label]= posterior[label]/sum(posterior.values())
  for label in posterior:
    
    if posterior[label] == max(list(posterior.values())):
      return label

In [478]:
def predict(X_test):
  y_pred = []
  for i in range(X_test.shape[0]):
    y_pred.append(naive_bayes(dict(X_test.iloc[i])))

  return y_pred

In [479]:
y_tst_pred = predict(X_test)
y_trn_pred = predict(X_train)

In [480]:
# from sklearn.metrics import accuracy_score
# accuracy_score(y_test, y_pred)

def accuracy(y_tst,y_prd):
  num_of_corr_lbl = 0

  for yt, yp in zip(y_tst, y_prd):
    if yt == yp:
      num_of_corr_lbl += 1
  acc = num_of_corr_lbl / len(y_tst)
  return acc

print('Training Accuracy',accuracy(y_trn_pred, y_train))

print('Test Accuracy',accuracy(y_tst_pred, y_test))

Training Accuracy 0.9200213751335946
Test Accuracy 0.9218812275807095
